# Exercise Sheet 8

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
# import tensorflow_probability as tfp
from keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Lambda, InputLayer, Flatten
from tensorflow.keras import Model

Using TensorFlow backend.


## Variational Autoencoder

The aim of this exercise is to implement the variational autoencoder and to test it on the polynomial dataset from the previous exercise sheet.  
* Implement the loss function of the variational autoencoder in your previous architecture.
* Compare the performance and the latent space representations of the traditional autoencoder and your variational autoencoder.
* $\star$ There is a quicker way of implementing the variational autoencoder using the tensorflow probability package. Using this package implement your variational autoencoder. As indicated, this exercise is optional.

### Solution

References from Monday's tutorial:  
[Deep learning in the heterotic orbifold landscape](https://arxiv.org/pdf/1811.05993.pdf)  
[Latent Constraints:
Learning to Generate Conditionally from Unconditional Generative Models](https://arxiv.org/pdf/1711.05772.pdf)

See also the [paper that introduced the VAEs](https://arxiv.org/abs/1312.6114)

We copy the data generation and autoencoder architecture from the last sheet

In [3]:
size = 40

# Evaluate polynomial over grid of size 40x40

def polynomial(degree):
    coeff = np.random.normal(0,1,(degree+1, degree+1))
    #coeff = np.random.uniform(-1,1,(degree+1, degree+1))
    return [[sum([coeff[i,j]*((x/size)**i)*((y/size)**j)
            for i in range(degree+1) for j in range(degree+1) if (i+j)<=degree]) 
            for x in range(size)] for y in range(size)]

# Degree two polynomials

Npoly = 3000
deg2polydata = np.array([polynomial(2) for i in range(Npoly)])
deg2polydata = deg2polydata.reshape(Npoly, size*size)
deg2mean = np.mean(deg2polydata)
deg2sdev = np.std(deg2polydata)
deg2polydata = tf.keras.utils.normalize(deg2polydata)
np.save('deg2polydata.npy', deg2polydata)

In [4]:
deg2polydata = np.load('deg2polydata.npy')
print(np.shape(deg2polydata))

(3000, 1600)


In [4]:
size = 40
input_size = size**2
hidden_size = 2
output_size = size**2

# Autoencoder with multiple hidden layers

# Encoder

AE2inputs = Input(shape=(input_size,))

AE2compress1 = Dense(512, activation='relu')(AE2inputs)
AE2compress2 = Dense(64, activation='relu')(AE2compress1)

AE2encoded = Dense(2, activation='linear')(AE2compress2)

# Decoder

AE2decompress1 = Dense(64, activation='relu')(AE2encoded)
AE2decompress2 = Dense(512, activation='relu')(AE2decompress1)
AE2decoded = Dense(size**2, activation='linear')(AE2decompress2)

AE2 = Model(AE2inputs, AE2decoded)

In [5]:
AE2.compile(optimizer='adam', loss='mse')

Number of epochs should be large in order to see the appearance of clusters. 200 is a good number that does not take much time on a GPU.

In [ ]:
hist_large = AE2.fit(deg2polydata, deg2polydata, epochs=200, batch_size=100)

Train on 3000 samples
Epoch 1/200
3000/3000 [==============================] - 5s 2ms/sample - loss: 4.9776e-04
Epoch 2/200
3000/3000 [==============================] - 1s 311us/sample - loss: 1.7467e-04
Epoch 3/200
3000/3000 [==============================] - 1s 271us/sample - loss: 1.0593e-04
Epoch 4/200
3000/3000 [==============================] - 1s 248us/sample - loss: 1.0171e-04
Epoch 5/200
3000/3000 [==============================] - 1s 411us/sample - loss: 1.0117e-04
Epoch 6/200
3000/3000 [==============================] - 1s 476us/sample - loss: 1.0138e-04
Epoch 7/200
3000/3000 [==============================] - 2s 594us/sample - loss: 1.0097e-04
Epoch 8/200
3000/3000 [==============================] - 1s 440us/sample - loss: 1.0108e-04
Epoch 9/200
3000/3000 [==============================] - 1s 469us/sample - loss: 1.0078e-04
Epoch 10/200
3000/3000 [==============================] - 1s 368us/sample - loss: 1.0047e-04
Epoch 11/200
3000/3000 [==============================] - 1

#### VAE using only Keras

Code partially adapted from [Keras Documentation](https://keras.io/examples/variational_autoencoder/)

In [ ]:
# Implement reparametriation trick

def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
# VAE loss function

def custom_loss(mean, var):
    def loss(y_true, y_pred):
        reconstruction_loss = tf.losses.mean_squared_error(y_true, y_pred)*size**2
        kl_loss = 1 + var - K.square(mean) - K.exp(var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        beta = 1.0*10**-2
        return K.mean(reconstruction_loss + beta*kl_loss)
    return loss

In [ ]:
# build VAE - we separate encoder and decoder

# Encoder

VAEinputs = Input(shape=(input_size,))
VAEcompress1 = Dense(512, activation='relu')(VAEinputs)
VAEcompress2 = Dense(64, activation='relu')(VAEcompress1)
VAEencoded_mean = Dense(2)(VAEcompress2)
VAEencoded_var = Dense(2)(VAEcompress2)
VAEencoded = Lambda(sampling, output_shape=(2,))([VAEencoded_mean, VAEencoded_var])
encoder = Model(VAEinputs, [VAEencoded_mean, VAEencoded_var, VAEencoded])

latent_inputs = Input(shape=(2,))

# Decoder

VAEdecompress1 = Dense(64, activation='relu')(latent_inputs)
VAEdecompress2 = Dense(512, activation='relu')(VAEdecompress1)
VAEdecoded = Dense(size**2, activation='linear')(VAEdecompress2)
decoder = Model(latent_inputs, VAEdecoded)

VAEoutputs = decoder(encoder(VAEinputs)[2])

VAE = Model(VAEinputs, VAEoutputs)

In [ ]:
VAE.compile(optimizer='adam', loss=custom_loss(VAEencoded_mean, VAEencoded_var))

In [ ]:

hist_VAE = VAE.fit(deg2polydata, deg2polydata, epochs=200, batch_size=100)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,5))

AE2_latent = Model(AE2inputs, AE2encoded)
AE2_encoded_data = AE2_latent.predict(deg2polydata)

VAE_encoded_data = encoder.predict(deg2polydata)[2]

axes[0].scatter(AE2_encoded_data[:,0], AE2_encoded_data[:,1], s=1)
axes[0].set_title('Autoencoder latent dimensions')
axes[1].scatter(VAE_encoded_data[:,0], VAE_encoded_data[:,1], s=1)
_ = axes[1].set_title('VAE latent dimensions')

The right hand side is clustered around the origin and looks much more Gaussian than the LHS

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,5))

polynumber = 20

axes[0].imshow(VAE.predict(deg2polydata)[polynumber].reshape(size,size))
axes[1].imshow(deg2polydata[polynumber].reshape(size,size))

We see that some samples are learned really well, up to some noise. Better results could probably achieved with more epochs and an annealing schedule for the relative importance of the reconstruction and KL losses.

#### VAE using TFP

See also [here](https://colab.research.google.com/github/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Probabilistic_Layers_VAE.ipynb)

In [ ]:
# Build VAE - we separate encoder and decoder

tfd = tfp.distributions
tfpl = tfp.layers

# Define the reference distribution
standardnormal = tfd.Independent(tfd.Normal(loc=tf.zeros(2), scale=1), reinterpreted_batch_ndims=1)

# Encoder

size = 40

VAEinputs = Input(shape=(1600,))
VAEcompress1 = Dense(512, activation='relu')(VAEinputs)
VAEcompress2 = Dense(64, activation='relu')(VAEcompress1)
# The purpose of the next layer is to shape the data in the right form for the MultivariateNormalTriL layer
# The output should be interpreted as the ``square root'' of the covariance matrix Out.Out^T=Cov
# Out is lower triangular. In our case this amounts to five parameters: 2 means, 3 matrix entries of Out
VAEencoded1 = Dense(tfpl.MultivariateNormalTriL.params_size(2))(VAEcompress2)
# The MultivariateNormalTriL layer does all the work. We have to pass the KL Divergence as a regularizer
VAEencoded2 = tfpl.MultivariateNormalTriL(2, activity_regularizer=tfpl.KLDivergenceRegularizer(standardnormal, weight=10**-5))(VAEencoded1)
encoder = Model(VAEinputs, VAEencoded2)

latent_inputs = Input(shape=(2,))

# Decoder

VAEdecompress1 = Dense(64, activation='relu')(latent_inputs)
VAEdecompress2 = Dense(512, activation='relu')(VAEdecompress1)
VAEdecoded = Dense(size**2, activation='linear')(VAEdecompress2)
decoder = Model(latent_inputs, VAEdecoded)

VAEoutputs = decoder(encoder(VAEinputs))

VAE = Model(VAEinputs, VAEoutputs)

In [ ]:
VAE.compile(optimizer='adam', loss='mse')

In [ ]:
hist_VAE = VAE.fit(deg2polydata, deg2polydata, epochs=100, batch_size=100)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,5))

polynumber = 3

axes[0].imshow(VAE.predict(deg2polydata)[polynumber].reshape(size,size))
axes[1].imshow(deg2polydata[polynumber].reshape(size,size))